In [1]:
%matplotlib inline
import pandas as pd
import os
import re
import shutil
import numpy as np
import warnings

In [2]:
df = pd.read_csv('students.csv', sep=';')

In [3]:
df

,family name,name
0,Falkhage,Cornelia
1,Fällman,Amanda
2,Feldt,Axel
3,Flodin,Samuel
4,Florin,Ida
5,Forssgren,Anna
6,Forssmed,Elsa
7,Fredriksson,Mathias
8,Fröberg,Vincent
9,Furborg,Sara


In [4]:
regexp=re.compile(r'^ *(.*)')
def clean_names(name):
    result=regexp.search(name)
    return result.group(1)

In [5]:
df['name']=df['name'].apply(func=clean_names)

In [6]:
def full_name(row):
    return '%s %s' % (row['name'],row['family name'])

In [7]:
df.index = df.apply(func=full_name, axis=1)
assert df.index.is_unique

In [8]:
def create_folder(row,root_path='assignments'):
    
    family_name = row['family name']
    name = row['name']
    dir_name = '%s_%s' % (family_name,name)
    path = os.path.abspath(os.path.join(root_path,dir_name))
    if not os.path.exists(path):
        os.mkdir(path)
    
    return path
    

## Create folders:

In [9]:
root_path='assignments'

if not os.path.exists(root_path):
    os.mkdir(root_path)

df['dir_path'] = df.apply(func=create_folder, axis=1, root_path=root_path)

In [10]:
def make_clickable(val):
    s=r'file:///' + val
    return '<a href="{}">{}</a>'.format(s,"path")

In [11]:
def show(df):
    return df.style.format({'dir_path': make_clickable})

## Sort the assignment files into each student folder
the files from Canvas look something like: *falkhagecornelia_63854_1658103_student_AutomaticGearbox.m*

In [12]:
df['code name'] = df.apply(lambda x:x['family name'].lower()+x['name'].lower(), axis=1)

In [13]:
file_name = 'falkhagecornelia_63854_1658103_student_AutomaticGearbox.m'
regexp = re.compile(pattern=r'([^_]+)_\d+_\d+_(.+)')
regexp.search(file_name).group(1)

'falkhagecornelia'

In [14]:
regexp.search(file_name).group(2)

'student_AutomaticGearbox.m'

In [15]:
submission_dir = 'submissions/'
files = os.listdir(submission_dir)
df_files = pd.DataFrame()

for file_name in files:
    s = pd.Series(dtype='object')
    result = regexp.search(file_name)
    s['code name'] = result.group(1)
    s['file name'] = result.group(2)
    s['original file name'] = file_name
    df_files = df_files.append(s, ignore_index=True)


In [16]:
df_files.head()

,code name,file name,original file name
0,ahagenalbin,DragsterAssignment_Albin_Ahagen.docx,ahagenalbin_61667_1658733_DragsterAssignment_A...
1,ahagenalbin,student_AutomaticGearbox.m,ahagenalbin_61667_1658734_student_AutomaticGea...
2,albertssontim,Rapport inlämningsuppgift 1.pdf,albertssontim_60948_1650158_Rapport inlämnings...
3,albertssontim,student_AutomaticGearbox.m,albertssontim_60948_1650159_student_AutomaticG...
4,albertssontim,student_AutomaticGearbox.asv,albertssontim_60948_1650160_student_AutomaticG...


In [17]:
df_files=pd.merge(left=df, right=df_files, how='outer', left_on='code name', right_on='code name').dropna(subset=['family name'])

In [18]:
df_files.tail()

,family name,name,dir_path,code name,file name,original file name
45,Hansson,Fredrik,E:\dev\TME136-Programming-and-algorithms\2021\...,hanssonfredrik,student_AutomaticGearbox.m,hanssonfredrik_61305_1652063_student_Automatic...
46,Hässel,Gustav,E:\dev\TME136-Programming-and-algorithms\2021\...,hässelgustav,bilhastighet-1.m,hässelgustav_57869_1653071_bilhastighet-1.m
47,Hässel,Gustav,E:\dev\TME136-Programming-and-algorithms\2021\...,hässelgustav,student_AutomaticGearbox.m,hässelgustav_57869_1653072_student_AutomaticGe...
48,Hässel,Gustav,E:\dev\TME136-Programming-and-algorithms\2021\...,hässelgustav,Inlämmning caster.pages,hässelgustav_57869_1653073_Inlämmning caster....
49,Helland,Wilmer,E:\dev\TME136-Programming-and-algorithms\2021\...,hellandwilmer,Automatic gearbox - Wilmer Helland.pdf,hellandwilmer_61398_1657862_Automatic gearbox ...


### copy the files for each student

In [19]:
def move_assignment_file_to_student_folder_with_correct_file_name(row, submission_dir):
    
    if pd.isnull(row['original file name']):
        return
    
    src=os.path.join(submission_dir, row['original file name'])
    dst=os.path.join(row['dir_path'],row['file name'])
    
    if not os.path.exists(dst):
        shutil.copyfile(src,dst)

In [20]:
df_files.apply(move_assignment_file_to_student_folder_with_correct_file_name, submission_dir=submission_dir, axis=1);

## Check exist: *student_AutomaticGearbox.m*

In [21]:
run_file_name='student_AutomaticGearbox.m'
def has_run_file(row, run_file_name='student_AutomaticGearbox.m'):
    path=os.path.join(row['dir_path'],run_file_name)
    return os.path.exists(path)

In [22]:
def recreate_run_file(row):
    files = os.listdir(row['dir_path'])
    for file in files:
        name,ext = os.path.splitext(file)
        if ext=='.m':
            if 'student_AutomaticGearbox' in file:
                src=os.path.join(row['dir_path'],file)
                dst=os.path.join(row['dir_path'],'student_AutomaticGearbox.m')
                if not os.path.exists(dst):
                    shutil.copyfile(src,dst)

In [23]:
df['has_run_file'] = df.apply(func=has_run_file, axis=1)

In [24]:
mask = ~df['has_run_file']
df.loc[mask].apply(func=recreate_run_file, axis=1)

Zahran Saeed Ghazy      None
Karl-Axel Gustafsson    None
Hans Hallbäck           None
Wilmer Helland          None
dtype: object

In [25]:

show(df.loc[mask])

,family name,name,dir_path,code name,has_run_file
Zahran Saeed Ghazy,Ghazy,Zahran Saeed,path,ghazyzahran saeed,False
Karl-Axel Gustafsson,Gustafsson,Karl-Axel,path,gustafssonkarl-axel,False
Hans Hallbäck,Hallbäck,Hans,path,hallbäckhans,False
Wilmer Helland,Helland,Wilmer,path,hellandwilmer,False


## Running the scripts to get the times

In [26]:
def runner(row, eng):
    
    print('evaluate: %s %s' % (row['name'],row['family name']))
    run_script_name = 'student_AutomaticGearbox.m'
    
    if os.path.exists(run_script_name):
        os.remove(run_script_name)  # Remove old.
    
    assert not os.path.exists(run_script_name)
    
    for file in os.listdir(row['dir_path']):
        ext = os.path.splitext(file)[-1]
        if ext=='.m' or ext=='.mat':
            src=os.path.join(row['dir_path'],file)
            dst=file
            shutil.copyfile(src,dst)
    
    assert os.path.exists(run_script_name)
    
    r = eng.runner()
    finishing_time=r.pop('finishing_time')
    result = r
    #finishing_time=result['TimeTotal'][0][-1]
    return finishing_time
    

This one got stuck in an infinite loop beacuse Throttle is not set. **"Warning: You aren't pressing the throttle. This will stop the car. A car that is stationary won't win dragraces. You
are probably watching the simulation stuck in an infinite loop right now "**

In [27]:
def manual(full_name:str, ok:bool, comment:str, finishing_time=None):
    df.loc[full_name,'ok'] = ok
    df.loc[full_name,'manual'] = True
    df.loc[full_name,'comment'] = comment
    df.loc[full_name,'finishing_time'] = finishing_time

In [28]:
#manual(full_name='Emelie Gillerstedt', ok=False, 
#       comment='This one got stuck in an infinite loop beacuse Throttle is not set')
#
#manual(full_name='Rasmus Fischer', ok=False, 
#       comment='Incomplete if clause if, elseif but no else')
#
#

In [29]:
import matlab.engine
eng = matlab.engine.start_matlab()

for index, row in df.iterrows():
    
    if 'manual' in row:
        if row['manual'] is True:
            print('(Handled manually)')
            continue  # This one is handled manually
    
    if 'ok' in row:
        if row['ok'] is True:
            print('(ok, already checked)')
            continue
    
    try:
        finishing_time = runner(row=row, eng=eng)
    except Exception as e:
        fail = True
        df.loc[index,'finishing_time'] = np.NaN
        print('Fail!')
    else:
        fail = False
        df.loc[index,'finishing_time'] = finishing_time
        df.loc[index,'ok'] = True
            
eng.quit()

evaluate: Cornelia Falkhage
evaluate: Amanda Fällman
evaluate: Axel Feldt
evaluate: Samuel Flodin
evaluate: Ida Florin
evaluate: Anna Forssgren
evaluate: Elsa Forssmed
evaluate: Mathias Fredriksson
evaluate: Vincent Fröberg
evaluate: Sara Furborg
evaluate: Hampus Gelang
evaluate: Zahran Saeed Ghazy
Fail!
evaluate: Carl Görling
evaluate: Matilda Graad
evaluate: Adam Gunnarsson
evaluate: Amanda Gustafsson
evaluate: Karl-Axel Gustafsson
Fail!
evaluate: Måns Gustafsson
evaluate: William Gustafsson
evaluate: Oskar Haapalo
evaluate: Hans Hallbäck
Fail!
evaluate: Gustav Haller
evaluate: Fredrik Hansson
evaluate: Gustav Hässel
evaluate: Wilmer Helland
Fail!


In [30]:
eng.quit()

In [31]:
show(df)

,family name,name,dir_path,code name,has_run_file,finishing_time,ok
Cornelia Falkhage,Falkhage,Cornelia,path,falkhagecornelia,True,15.531188,True
Amanda Fällman,Fällman,Amanda,path,fällmanamanda,True,15.263195,True
Axel Feldt,Feldt,Axel,path,feldtaxel,True,15.263195,True
Samuel Flodin,Flodin,Samuel,path,flodinsamuel,True,15.261506,True
Ida Florin,Florin,Ida,path,florinida,True,15.261506,True
Anna Forssgren,Forssgren,Anna,path,forssgrenanna,True,16.978942,True
Elsa Forssmed,Forssmed,Elsa,path,forssmedelsa,True,16.978942,True
Mathias Fredriksson,Fredriksson,Mathias,path,fredrikssonmathias,True,15.271930,True
Vincent Fröberg,Fröberg,Vincent,path,fröbergvincent,True,15.271930,True
Sara Furborg,Furborg,Sara,path,furborgsara,True,15.261515,True


In [36]:
show(df.loc[df['ok']==True].sort_values(by='family name'))

,family name,name,dir_path,code name,has_run_file,finishing_time,ok
Cornelia Falkhage,Falkhage,Cornelia,path,falkhagecornelia,True,15.531188,True
Axel Feldt,Feldt,Axel,path,feldtaxel,True,15.263195,True
Samuel Flodin,Flodin,Samuel,path,flodinsamuel,True,15.261506,True
Ida Florin,Florin,Ida,path,florinida,True,15.261506,True
Anna Forssgren,Forssgren,Anna,path,forssgrenanna,True,16.978942,True
Elsa Forssmed,Forssmed,Elsa,path,forssmedelsa,True,16.978942,True
Mathias Fredriksson,Fredriksson,Mathias,path,fredrikssonmathias,True,15.271930,True
Vincent Fröberg,Fröberg,Vincent,path,fröbergvincent,True,15.271930,True
Sara Furborg,Furborg,Sara,path,furborgsara,True,15.261515,True
Amanda Fällman,Fällman,Amanda,path,fällmanamanda,True,15.263195,True


In [32]:
show(df.loc[df['ok']!=True])

,family name,name,dir_path,code name,has_run_file,finishing_time,ok
Zahran Saeed Ghazy,Ghazy,Zahran Saeed,path,ghazyzahran saeed,False,nan,nan
Karl-Axel Gustafsson,Gustafsson,Karl-Axel,path,gustafssonkarl-axel,False,nan,nan
Hans Hallbäck,Hallbäck,Hans,path,hallbäckhans,False,nan,nan
Wilmer Helland,Helland,Wilmer,path,hellandwilmer,False,nan,nan


In [33]:
df.sort_values(by='finishing_time', inplace=True)
show(df)

,family name,name,dir_path,code name,has_run_file,finishing_time,ok
Gustav Haller,Haller,Gustav,path,hallergustav,True,15.064779,True
Oskar Haapalo,Haapalo,Oskar,path,haapalooskar,True,15.064779,True
Amanda Gustafsson,Gustafsson,Amanda,path,gustafssonamanda,True,15.168952,True
Adam Gunnarsson,Gunnarsson,Adam,path,gunnarssonadam,True,15.168952,True
Samuel Flodin,Flodin,Samuel,path,flodinsamuel,True,15.261506,True
Ida Florin,Florin,Ida,path,florinida,True,15.261506,True
Hampus Gelang,Gelang,Hampus,path,gelanghampus,True,15.261515,True
Sara Furborg,Furborg,Sara,path,furborgsara,True,15.261515,True
Amanda Fällman,Fällman,Amanda,path,fällmanamanda,True,15.263195,True
Axel Feldt,Feldt,Axel,path,feldtaxel,True,15.263195,True


In [34]:
df.to_csv('Martin_Alexandersson_finishing_times.csv', sep=';', encoding='cp1252')